In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType,DoubleType,DecimalType
from pyspark.sql.functions import * 

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext

sc = SparkContext.getOrCreate()
gc = GlueContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#CREATE DATAFRAME WITH NULL VALUES
source_df = gc.create_data_frame_from_catalog( 
    database = "db_null_values", 
    table_name = "invoices_null_values", 
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.show(32)

In [5]:
# SET DECIMAL PLACES
costing_rate="costing rate"
format_costing_rate = format_number(round(costing_rate ,4), 4)
amount_due="amount due"
format_amount_due=format_number(round(amount_due ,2), 2)

source_df = source_df.withColumn(costing_rate, format_costing_rate) \
# amount due gives null value - not sure why

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.show()

In [7]:
# ADD YEAR TO DATE COLUMNS
due_date="due date"
issue_date="issue date"
year = F.lit("-2022")
due_date_add_year = F.concat(col(due_date),year)
issue_date_add_year = F.concat(col(issue_date),year)
                             
                             
# source_df = source_df.withColumn("due date", F.concat(col("due date"),F.lit("-2022"))) \
#     .withColumn("issue date", F.concat(col("issue date"),F.lit("-2022")))
                               
source_df = source_df.withColumn(due_date, due_date_add_year) \
    .withColumn(issue_date, issue_date_add_year)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
source_df.show()

In [8]:
# FORMAT DATES
source_df = source_df.select(
    col("external reference"), \
    col("number"), \
    col("counterparty"), \
    col("amount due"), \
    col("currency"), \
    col("cost currency"), \
    col("costing rate"), \
    to_date(F.col("due date"), "d-MMM-yyy").alias("due date"), \
    to_date(F.col("issue date"), "d-MMM-yyy").alias("issue date")) \

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
source_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------+--------------------+-----------+--------+-------------+------------+----------+----------+
|external reference|number|        counterparty| amount due|currency|cost currency|costing rate|  due date|issue date|
+------------------+------+--------------------+-----------+--------+-------------+------------+----------+----------+
|          SC25505A|137113|   Fruit Traders LLC| 10,800.00 |     USD|          ZAR|     15.8620|2022-01-24|2022-01-03|
|          SC25506A|137114|   Fruit Traders LLC| 10,800.00 |     USD|          ZAR|     15.8620|2022-01-24|2022-01-03|
|          SC25503A|137116|   Fruit Traders LLC| 10,800.00 |     USD|          ZAR|     15.8620|2022-01-24|2022-01-03|
|          SC25504A|137117|   Fruit Traders LLC| 10,800.00 |     USD|          ZAR|     15.8620|2022-01-24|2022-01-03|
|          SC25499A|137078|   Fruit Traders LLC| 10,134.00 |     USD|          ZAR|     15.8620|2022-01-24|2022-01-03|
|          SC25500A|137079|   Fruit Traders LLC|